# Self-play record viewer

- Records live under `logs/selfplay_records/`
- Use `show_game(iteration, index)` or run the input cell below.

Example: `show_game(71, 25)`


In [ ]:
import glob
import json
import os

RECORD_DIR = "logs/selfplay_records"
if not os.path.isdir(RECORD_DIR):
    if os.path.basename(os.getcwd()) == "selfplay_records":
        RECORD_DIR = "."

def _find_record_file(iteration, record_dir=RECORD_DIR):
    pattern = f"selfplay_*_iter_{iteration:04d}.json"
    candidates = glob.glob(os.path.join(record_dir, pattern))
    if not candidates:
        raise FileNotFoundError(f"No record file for iteration {iteration} in {record_dir}")
    candidates.sort(key=lambda p: os.path.getmtime(p))
    return candidates[-1]

def load_records(iteration, record_dir=RECORD_DIR):
    path = _find_record_file(iteration, record_dir)
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data.get("records", []), data.get("meta", {}), path

def _render_board(record):
    size = int(record.get("board_size", 0))
    board = [["." for _ in range(size)] for _ in range(size)]
    if record.get("center_wall") and size > 0:
        mid = size // 2
        board[mid][mid] = "#"
    for r, c in record.get("black", []):
        board[int(r)][int(c)] = "X"
    for r, c in record.get("white", []):
        board[int(r)][int(c)] = "O"

    cols = [chr(ord('A') + i) for i in range(size)]
    print("   " + " ".join(cols))
    for r in range(size):
        row_label = str(r + 1).rjust(2)
        print(f"{row_label} " + " ".join(board[r]))

def show_game(iteration, index, record_dir=RECORD_DIR):
    records, meta, path = load_records(iteration, record_dir)
    if index < 1 or index > len(records):
        raise IndexError(f"index out of range: 1..{len(records)}")
    record = records[index - 1]
    print(f"File: {path}")
    if meta:
        print(f"Iteration: {meta.get('iteration')} | Games: {meta.get('games')} | Record interval: {meta.get('record_interval')}")
    print(f"Winner: {record.get('winner_label')} | Win type: {record.get('win_type')} | Moves: {record.get('moves')} | Territory diff: {record.get('territory_diff')}")
    _render_board(record)


In [ ]:
# --- Input cell ---
# You can type either: 71,25  or  (71,25)
raw = input("Enter (iteration,index) e.g. 71,25 : ").strip()
raw = raw.strip('()')
parts = [p.strip() for p in raw.split(',') if p.strip()]
if len(parts) != 2:
    raise ValueError("Please enter two numbers: iteration,index")
iteration = int(parts[0])
index = int(parts[1])
show_game(iteration, index)
